## Data Reformatting (brat to ConLL)

In [1]:
# from google.colab import drive

# drive.mount('/content/drive')

In [2]:
path = '/scratch/js12684/final/'

In [3]:
#retrieve paths for text and ann data

import glob
notes_orig =glob.glob(path + 'data/*.txt')
ann_orig = glob.glob(path + 'data/*.ann' )

print(len(notes_orig))
print(len(ann_orig))

400
400


In [4]:
!mkdir '/scratch/js12684/final/ann_ref_disp'

mkdir: cannot create directory '/scratch/js12684/final/ann_ref_disp': File exists


In [5]:
#reformat BRAT annotation to CoNLL for drug mentions:

for file in ann_orig:
    with open(file, 'r') as fin:
        outfile = path + 'ann_ref_disp/' + file.split('/')[-1]
        with open(outfile, 'w') as fileout:
            for line in fin:
                if line.startswith('T'):
                    # newline = line.replace('NoDisposition', 'Drug').replace('Disposition', 'Drug').replace('Temporality', 'Drug').replace('Undetermined', "Drug")
                    fileout.write(line)

In [6]:
ann_disp = glob.glob(path + 'ann_ref_disp/*.ann')
print(len(ann_disp))

400


In [7]:
#convert reformatted ann file into tab separated ConLL format
import numpy as np
from urllib.parse import ParseResult
from urllib.parse import ParseResultBytes

outfile = path + '/output_disp.txt'
with open(outfile, 'w') as output_file:
  out_list = []
  for ann_file in ann_disp:
    with open(ann_file, 'r') as ann_in:
      txt_file = path + f'data/{ann_file[-10:-4]}.txt'
      with open(txt_file, 'r') as txt_in:
        start_end = []
        count = 0
        disp = {}
        
        #reading each line of .ann file and extracting start and end indices 
        for line in ann_in:
            entry = line.split()
            disp[count] = entry[1]
            count += 1
            start_end.append([int(entry[2]), int(entry[3])])
            
        #reading each line of .txt file and extracting drug names using list of indices
        text_tokens = txt_in.read().split()
        drug_names = []
        for indicies in start_end:
          start = indicies[0]
          end = indicies[1]

          with open(txt_file, 'r') as txt_in:
            drug_names.append(txt_in.read()[indicies[0]:indicies[1]])
        
        #appending "O" or Disp label to each token in list of txt tokens

        for token in text_tokens:
          token_alnum = ''.join(ch for ch in token if ch.isalnum())
          if token_alnum in drug_names:
            idx = drug_names.index(token_alnum)
            if token == text_tokens[0]:
              out_list.append(f'{ann_file[-10:-4]}\t')
            else:
              out_list.append(f'{np.nan}\t')
            out_list.append(f'{token}\t{disp[idx]}\n')
          else:
            if token == text_tokens[0]:
              out_list.append(f'{ann_file[-10:-4]}\t')
            else:
              out_list.append(f'{np.nan}\t')
            out_list.append(f'{token}\tO\n')
        out_list.append('\n')
  out_string = ''.join(out_list)
  output_file.write(out_string)

## Data Processing for SimpleTransformers

In [8]:
#read in ConLL annotation file as Csv
import pandas as pd

data = pd.read_csv(path + 'output_disp.txt', sep='\t', lineterminator='\n', names = ['Note', 'Token', 'Label'])

#fill missing note numbers
data = data.fillna(method = 'ffill')

In [9]:
token_count = []
for note in data['Note'].unique():
  df = data.loc[data['Note']== note]
  count = 0
  for token in df['Token']:
    token_count.append(count)
    count += 1

data['Token Count'] = token_count
data.sort_values(by = ['Note', 'Token Count'], inplace=True)
data.reset_index(drop = True, inplace = True)

In [10]:
#encode note number
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
note_en = le.fit_transform(data['Note'])

data['Note'] = note_en

In [11]:
#Convert column names to be compatible with model
data.rename(columns = {'Note':'sentence_id', 'Token':'words', 'Label':'labels'}, inplace=True)

In [12]:
#train test split
from sklearn.model_selection import train_test_split

# X = data[['sentence_id','words']]
# y = data['labels'] 

train, test = train_test_split(data, test_size = 0.2, shuffle=False, random_state = 42 )
train, val = train_test_split(train, test_size = 0.125, shuffle=False, random_state = 42 )

print(train.shape)
print(val.shape)
print(test.shape)

(171901, 4)
(24558, 4)
(49115, 4)


In [13]:
#building up train data and test data
train_data = pd.DataFrame({"sentence_id":train["sentence_id"],"words":train["words"],"labels":train['labels']})
val_data = pd.DataFrame({"sentence_id":val["sentence_id"],"words":val["words"],"labels":val['labels']})
test_data = pd.DataFrame({"sentence_id":test["sentence_id"],"words":test["words"],"labels":test['labels']})

In [14]:
label = data["labels"].unique().tolist()
label

['O', 'NoDisposition', 'Disposition', 'Undetermined']

In [15]:
train_data

,sentence_id,words,labels
0,0,Record,O
1,0,date:,O
2,0,2106-02-12,O
3,0,Campbell,O
4,0,Orthopedic,O
...,...,...,...
171896,276,"33.8,",O
171897,276,PLT,O
171898,276,601,O
171899,276,(H),O


## Model Setup

In [16]:
#label counts in train data
train_data['labels'].value_counts()

O                167754
NoDisposition      3043
Disposition         850
Undetermined        254
Name: labels, dtype: int64

In [17]:
#calculate weights for class imbalance
n_samples = len(train_data)
n_classes = 4
freq_O = train_data['labels'].value_counts()[0]
freq_nodisp = train_data['labels'].value_counts()[1]
freq_disp = train_data['labels'].value_counts()[2]
freq_und = train_data['labels'].value_counts()[3]

freq_list = [freq_O, freq_nodisp, freq_disp, freq_und]

weights = [n_samples/(n_classes * freq) for freq in freq_list]
print(weights)

[0.2561801805024023, 14.122658560630956, 50.55911764705883, 169.1938976377953]


In [18]:
# #wandb for model parameter visualization
# import wandb
# wandb.login()

In [19]:
# # model parameters
# from simpletransformers.ner import NERModel,NERArgs

# args = NERArgs()
# args.learning_rate = 5e-5
# args.scheduler = 'constant_schedule'
# args.num_train_epochs = 10
# args.train_batch_size = 32
# args.eval_batch_size = 32
# # args.logging_steps = 10
# # args.evaluate_during_training_steps = 2137
# # args.evaluate_during_training = True
# # args.evaluate_during_training_verbose = True
# args.overwrite_output_dir = True
# # args.use_cached_eval_features = True
# args.manual_seed = 42


## clinical BERT with weighted classes

In [26]:
from simpletransformers.ner import NERModel,NERArgs

def clinicalbert_hptune(lr, bs):
    # model parameters
    args = NERArgs()
    args.learning_rate = lr
    args.scheduler = 'constant_schedule'
    args.num_train_epochs = 10
    args.train_batch_size = bs
    args.eval_batch_size = bs
    # args.logging_steps = 10
    # args.evaluate_during_training_steps = 2137
    # args.evaluate_during_training = True
    # args.evaluate_during_training_verbose = True
    args.overwrite_output_dir = True
    # args.use_cached_eval_features = True
    args.manual_seed = 42
    
    #instantiate model
    cb_model = NERModel('bert', 'emilyalsentzer/Bio_ClinicalBERT', labels = label ,args = args ,weight = weights, use_cuda=True)
    
    #train and eval model with train + val data
    cb_model.train_model(train_data, eval_data = val_data)
    
    #test model on test data
    cb_result, cb_model_outputs, cb_preds_list = cb_model.eval_model(test_data)
    
    print(cb_result)
    
    cb_model.predict(['As a result of this, I think it is reasonable for us in addition to having her on atenolol to stop the hydrochlorothiazide, put her on ramipril and a nitrate.'])

In [28]:
args = NERArgs()
args.learning_rate = 5e-5
args.scheduler = 'constant_schedule'
args.num_train_epochs = 10
args.train_batch_size = 32
args.eval_batch_size = 32
# args.logging_steps = 10
# args.evaluate_during_training_steps = 2137
# args.evaluate_during_training = True
# args.evaluate_during_training_verbose = True
args.overwrite_output_dir = True
# args.use_cached_eval_features = True
args.manual_seed = 42

In [29]:
#instantiate model
cb_model = NERModel('bert', 'emilyalsentzer/Bio_ClinicalBERT', labels = label ,args = args ,weight = weights, use_cuda=True)

#train and eval model with train + val data
cb_model.train_model(train_data, eval_data = val_data)

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

  0%|          | 0/1 [00:00<?, ?it/s]

/home/js12684/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

...

Running Epoch 6 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

Running Epoch 7 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

(90, 0.2169039698705698)

In [30]:
#test model on test data
cb_result, cb_model_outputs, cb_preds_list = cb_model.eval_model(test_data)

print(cb_result)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 0.68927334745725, 'precision': 0.7021276595744681, 'recall': 0.717391304347826, 'f1_score': 0.7096774193548387}


/home/js12684/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NoDisposition seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/js12684/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Disposition seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


In [31]:
cb_model.predict(['As a result of this, I think it is reasonable for us in addition to having her on atenolol to stop the hydrochlorothiazide, put her on ramipril and a nitrate.'])

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

([[{'As': 'O'},
   {'a': 'O'},
   {'result': 'O'},
   {'of': 'O'},
   {'this,': 'O'},
   {'I': 'O'},
   {'think': 'O'},
   {'it': 'O'},
   {'is': 'O'},
   {'reasonable': 'O'},
   {'for': 'O'},
   {'us': 'O'},
   {'in': 'O'},
   {'addition': 'O'},
   {'to': 'O'},
   {'having': 'O'},
   {'her': 'O'},
   {'on': 'O'},
   {'atenolol': 'NoDisposition'},
   {'to': 'O'},
   {'stop': 'O'},
   {'the': 'O'},
   {'hydrochlorothiazide,': 'NoDisposition'},
   {'put': 'O'},
   {'her': 'O'},
   {'on': 'O'},
   {'ramipril': 'NoDisposition'},
   {'and': 'O'},
   {'a': 'O'},
   {'nitrate.': 'NoDisposition'}]],
 [[{'As': [[5.844, -1.1045, -2.975, -2.002]]},
   {'a': [[5.645, -1.271, -2.613, -1.942]]},
   {'result': [[5.24, -1.228, -2.498, -1.49]]},
   {'of': [[5.598, -1.591, -2.377, -1.567]]},
   {'this,': [[5.918, -1.381, -2.527, -1.619], [5.95, -1.433, -2.81, -1.812]]},
   {'I': [[5.97, -1.382, -3.035, -2.117]]},
   {'think': [[5.914, -1.601, -3.057, -1.648]]},
   {'it': [[5.895, -1.146, -2.86, -1.775]]

In [41]:
[[{'As': 'O'},
   {'a': 'O'},
   {'result': 'O'},
   {'of': 'O'},
   {'this,': 'O'},
   {'I': 'O'},
   {'think': 'O'},
   {'it': 'O'},
   {'is': 'O'},
   {'reasonable': 'O'},
   {'for': 'O'},
   {'us': 'O'},
   {'in': 'O'},
   {'addition': 'O'},
   {'to': 'O'},
   {'having': 'O'},
   {'her': 'O'},
   {'on': 'O'},
   {'atenolol': 'NoDisposition'},
   {'to': 'O'},
   {'stop': 'O'},
   {'the': 'O'},
   {'hydrochlorothiazide,': 'Disposition'},
   {'put': 'O'},
   {'her': 'O'},
   {'on': 'O'},
   {'ramipril': 'Disposition'},
   {'and': 'O'},
   {'a': 'O'},
   {'nitrate.': 'Disposition'}]]

[[{'As': 'O'},
  {'a': 'O'},
  {'result': 'O'},
  {'of': 'O'},
  {'this,': 'O'},
  {'I': 'O'},
  {'think': 'O'},
  {'it': 'O'},
  {'is': 'O'},
  {'reasonable': 'O'},
  {'for': 'O'},
  {'us': 'O'},
  {'in': 'O'},
  {'addition': 'O'},
  {'to': 'O'},
  {'having': 'O'},
  {'her': 'O'},
  {'on': 'O'},
  {'atenolol': 'NoDisposition'},
  {'to': 'O'},
  {'stop': 'O'},
  {'the': 'O'},
  {'hydrochlorothiazide,': 'Disposition'},
  {'put': 'O'},
  {'her': 'O'},
  {'on': 'O'},
  {'ramipril': 'Disposition'},
  {'and': 'O'},
  {'a': 'O'},
  {'nitrate.': 'Disposition'}]]

### Learning rate 5e-4, BS 8

In [25]:
clinicalbert_hptune(5e-4, 8)

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

  0%|          | 0/1 [00:00<?, ?it/s]

/home/js12684/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/35 [00:00<?, ?it/s]

...

Running Epoch 9 of 10:   0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 1.0064942598342896, 'precision': 0.0, 'recall': 0.0, 'f1_score': 0.0}


/home/js12684/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NoDisposition seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/js12684/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Disposition seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/js12684/.local/lib/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Learning rate 5e-4, BS 16

In [26]:
clinicalbert_hptune(5e-4, 16)

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

  0%|          | 0/1 [00:00<?, ?it/s]

/home/js12684/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/18 [00:00<?, ?it/s]

...

Running Epoch 9 of 10:   0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 1.0409679770469666, 'precision': 0.0, 'recall': 0.0, 'f1_score': 0.0}


/home/js12684/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NoDisposition seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/js12684/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Disposition seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/js12684/.local/lib/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Learning rate 5e-4, BS 32

In [27]:
clinicalbert_hptune(5e-4, 32)

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

  0%|          | 0/1 [00:00<?, ?it/s]

/home/js12684/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

...

Running Epoch 9 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 1.7186731894810994, 'precision': 0.0, 'recall': 0.0, 'f1_score': 0.0}


/home/js12684/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NoDisposition seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/js12684/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Disposition seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


### Learning rate 1e-4, BS 8

In [28]:
clinicalbert_hptune(1e-4, 8)

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

  0%|          | 0/1 [00:00<?, ?it/s]

/home/js12684/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/35 [00:00<?, ?it/s]

...

Running Epoch 9 of 10:   0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 1.190849978326878, 'precision': 0.5918367346938775, 'recall': 0.6304347826086957, 'f1_score': 0.6105263157894736}


/home/js12684/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NoDisposition seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/js12684/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Disposition seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


### Learning rate 1e-4, BS 16

In [19]:
clinicalbert_hptune(1e-4, 16)

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

  0%|          | 0/1 [00:00<?, ?it/s]

/home/js12684/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/18 [00:00<?, ?it/s]

...

Running Epoch 9 of 10:   0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 1.493959802389145, 'precision': 0.6904761904761905, 'recall': 0.6304347826086957, 'f1_score': 0.6590909090909092}


/home/js12684/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NoDisposition seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/js12684/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Disposition seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


### Learning rate 5e-5, BS 8

In [30]:
clinicalbert_hptune(5e-5, 8)

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

  0%|          | 0/1 [00:00<?, ?it/s]

/home/js12684/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/35 [00:00<?, ?it/s]

...

Running Epoch 9 of 10:   0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 1.43701814111555, 'precision': 0.6, 'recall': 0.5869565217391305, 'f1_score': 0.5934065934065934}


/home/js12684/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NoDisposition seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/js12684/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Disposition seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


### Learning rate 5e-5, BS 16

In [31]:
clinicalbert_hptune(5e-5, 16)

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

  0%|          | 0/1 [00:00<?, ?it/s]

/home/js12684/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/18 [00:00<?, ?it/s]

...

Running Epoch 9 of 10:   0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 1.0660235379822551, 'precision': 0.5740740740740741, 'recall': 0.6739130434782609, 'f1_score': 0.62}


/home/js12684/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NoDisposition seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/js12684/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Disposition seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


### Learning rate 5e-5, BS 32

In [21]:
clinicalbert_hptune(5e-5, 32)

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

  0%|          | 0/1 [00:00<?, ?it/s]

/home/js12684/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

...

Running Epoch 9 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 0.68927334745725, 'precision': 0.7021276595744681, 'recall': 0.717391304347826, 'f1_score': 0.7096774193548387}


/home/js12684/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NoDisposition seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/js12684/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Disposition seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


In [23]:
cb_model.predict['As a result of this, I think it is reasonable for us in addition to having her on atenolol to stop the hydrochlorothiazide, put her on ramipril and a nitrate.']

NameError: name 'cb_model' is not defined

### Learning rate 1e-5, BS 8

In [33]:
clinicalbert_hptune(1e-5, 8)

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

  0%|          | 0/1 [00:00<?, ?it/s]

/home/js12684/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/35 [00:00<?, ?it/s]

...

Running Epoch 9 of 10:   0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 0.6101155424490571, 'precision': 0.45, 'recall': 0.5869565217391305, 'f1_score': 0.5094339622641509}


/home/js12684/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NoDisposition seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/js12684/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Disposition seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


### Learning rate 1e-5, BS 16

In [34]:
clinicalbert_hptune(1e-5, 16)

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

  0%|          | 0/1 [00:00<?, ?it/s]

/home/js12684/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/18 [00:00<?, ?it/s]

...

Running Epoch 9 of 10:   0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 0.7191230248659849, 'precision': 0.5370370370370371, 'recall': 0.6304347826086957, 'f1_score': 0.5800000000000001}


/home/js12684/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NoDisposition seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/js12684/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Disposition seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


### Learning rate 1e-5, BS 32

In [35]:
clinicalbert_hptune(1e-5, 32)

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

  0%|          | 0/1 [00:00<?, ?it/s]

/home/js12684/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

...

Running Epoch 9 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 0.4576147397359212, 'precision': 0.4745762711864407, 'recall': 0.6086956521739131, 'f1_score': 0.5333333333333333}


/home/js12684/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NoDisposition seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/js12684/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Disposition seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


### Learning rate 5e-6, BS 8

In [36]:
clinicalbert_hptune(5e-6, 8)

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

  0%|          | 0/1 [00:00<?, ?it/s]

/home/js12684/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/35 [00:00<?, ?it/s]

...

Running Epoch 9 of 10:   0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 0.6352067492902279, 'precision': 0.48214285714285715, 'recall': 0.5869565217391305, 'f1_score': 0.5294117647058824}


/home/js12684/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NoDisposition seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/js12684/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Disposition seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


### Learning rate 5e-6, BS 16

In [37]:
clinicalbert_hptune(5e-6, 16)

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

  0%|          | 0/1 [00:00<?, ?it/s]

/home/js12684/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/18 [00:00<?, ?it/s]

...

Running Epoch 9 of 10:   0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 0.49084227383136747, 'precision': 0.39344262295081966, 'recall': 0.5217391304347826, 'f1_score': 0.4485981308411215}


/home/js12684/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NoDisposition seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/js12684/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Disposition seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


### Learning rate 5e-6, BS 32

In [38]:
clinicalbert_hptune(5e-6, 32)

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

  0%|          | 0/1 [00:00<?, ?it/s]

/home/js12684/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

...

Running Epoch 9 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 0.4572124779224396, 'precision': 0.3064516129032258, 'recall': 0.41304347826086957, 'f1_score': 0.35185185185185186}


/home/js12684/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NoDisposition seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/js12684/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Disposition seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


### Learning rate 1e-6, BS 8

In [39]:
clinicalbert_hptune(1e-6, 8)

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

  0%|          | 0/1 [00:00<?, ?it/s]

/home/js12684/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/35 [00:00<?, ?it/s]

...

Running Epoch 9 of 10:   0%|          | 0/35 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 0.6520083576440812, 'precision': 0.5, 'recall': 0.32608695652173914, 'f1_score': 0.39473684210526316}


/home/js12684/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NoDisposition seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/js12684/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Disposition seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


### Learning rate 1e-6, BS 16

In [40]:
clinicalbert_hptune(1e-6, 16)

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

  0%|          | 0/1 [00:00<?, ?it/s]

/home/js12684/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/18 [00:00<?, ?it/s]

...

Running Epoch 9 of 10:   0%|          | 0/18 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 0.9095423698425293, 'precision': 0.24, 'recall': 0.13043478260869565, 'f1_score': 0.16901408450704222}


/home/js12684/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NoDisposition seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/js12684/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Disposition seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


### Learning rate 1e-6, BS 32

In [41]:
clinicalbert_hptune(1e-6, 32)

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint 

  0%|          | 0/1 [00:00<?, ?it/s]

/home/js12684/.local/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

...

Running Epoch 9 of 10:   0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/3 [00:00<?, ?it/s]

{'eval_loss': 1.1061046918233235, 'precision': 0.048, 'recall': 0.13043478260869565, 'f1_score': 0.07017543859649122}


/home/js12684/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NoDisposition seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/js12684/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Disposition seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/js12684/.local/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: Undetermined seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
